解决线程同步可以通过以下方式：
- 线程锁Lock和Rlock
- 条件变量condition
- 信号量Semaphore

# 1、线程锁—Lock和RLock

In [21]:
total = 0

def add():
    global total
    for i in range(1000000):
        total += 1

def desc():
    global total
    for i in range(1000000):
        total -= 1
        
import threading
thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=desc)
thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(total)

390913


- 上面的程序为什么每次输出的值不一样？而且不是0？

> Python代码执行的时候是转化成字节码执行的，一个函数回转化成多行字节码。多个线程在切换的时候是根据字节码的行数或者时间片切换的，这就导致一个函数的还没有执行完，线程就会切换到另一个的字节码去执行，同时另一个函数的字节码也没执行完，就又会切换。所以会导致变量赋值的时候出错。

## 如何解决？

In [5]:
# 通过锁来解决线程同步的问题
from threading import Lock

total = 0
lock = Lock()  # 共用一把锁
def add():
    global total
    global lock
    for i in range(1000000):
        lock.acquire() # 获得锁
        total += 1
        lock.release() # 释放锁

def desc():
    global total
    global lock
    for i in range(1000000):
        lock.acquire()
        total -= 1
        lock.release()
        
import threading
thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=desc)
thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(total)

0


锁一定程度上可以解线程同步的问题，但是也有缺点：

- 获取锁和释放锁的过程是需要等待的，这个过程比较耗时。

- 有可能造成死锁。
    如果连续两次调用lock.acquire()会造成死锁。
    如：
    ```Python
    def A():
        lock.acquire()
        B()
        # do something
        lock.release()

    def B():
        lock.acquire()
        # do something
        lock.release()
    ```

为了解决上面的问题，Python还提供了另一个锁RLock(可重入的锁)，在**同一个线程**里面可以连续调用多次acquire()，但是要注意，acquire()的次数要和release()的次数相等。

# 2、条件变量Condition
用于复杂的线程间同步

In [20]:
# 实现小爱同学和天猫精灵对话，一人一句

from threading import Thread, Condition

class XiaoAi(Thread):
    def __init__(self, cond):
        super().__init__(name='小爱')
        self.cond = cond
        
    def run(self):
        with self.cond:
            self.cond.wait()  # 最开始是天猫精灵先说话的，所以此时小爱需要等待
            
            print('{}: 在'.format(self.name))
            self.cond.notify() # 小爱说完后通知天猫精灵说话
            self.cond.wait() # 自己等待
            
            print('{}: 好啊'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 君住长江尾'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 共饮长江水'.format(self.name))
            self.cond.notify()
            self.cond.wait()

            print('{}: 此恨何时已'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 定不负相思意'.format(self.name))
            self.cond.notify()
            self.cond.wait()


class TianMao(Thread):
    def __init__(self, cond):
        super().__init__(name='天猫精灵')
        self.cond = cond
        
    def run(self):
        with self.cond:
            print('{}: 小爱同学'.format(self.name))
            self.cond.notify() # 通知小爱说话
            self.cond.wait() # 自己开始等待
            
            print('{}: 我们来对古诗吧'.format(self.name))
            self.cond.notify()
            self.cond.wait()

            print('{}: 我住长江头'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 日日思君不见君'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 此水几时休'.format(self.name))
            self.cond.notify()
            self.cond.wait()
            
            print('{}: 只愿君心似我心'.format(self.name))
            self.cond.notify()
            self.cond.wait()


cond = Condition()
xiaoai = XiaoAi(cond)
tianmao = TianMao(cond)

# 调用Condition的时候启动顺序很重要
# 如果天猫先启动的话，他第一次通知小爱的时候（notify），小爱还没有启动，根本就接收不到天猫的通知
# 而此时天猫精灵已经进入了等到（wait），但是wait()方法只能通过notify()来唤醒，也就是说只能让小爱通过notify()来唤醒天猫
# 但此时小爱没启动啊，所以就陷入了僵局

xiaoai.start()
tianmao.start()

天猫精灵: 小爱同学
小爱: 在
天猫精灵: 我们来对古诗吧
小爱: 好啊
天猫精灵: 我住长江头
小爱: 君住长江尾
天猫精灵: 日日思君不见君
小爱: 共饮长江水
天猫精灵: 此水几时休
小爱: 此恨何时已
天猫精灵: 只愿君心似我心
小爱: 定不负相思意


# 3、Semaphore
用于控制进入数量的锁

场景：爬虫的时候可以控制请求的并发量

In [28]:
import threading
import time

class HtmlSpider(threading.Thread):
    def __init__(self, url, sem):
        super().__init__()
        self.url = url
        self.sem = sem
    
    def run(self):
        time.sleep(2)
        print('got html text success')
        self.sem.release() # 没调用一次release()，num就会加一
        
class UrlProducer(threading.Thread):
    def __init__(self, sem):
        super().__init__()
        self.sem = sem
    
    def run(self):
        for i in range(20):
            self.sem.acquire() # 每调用一次acquire()，num就会减一
            html_thread = HtmlSpider('https://baidu.com/{}'.format(i), sem)
            html_thread.start()
            
            
# 每次只允许3个并发            
num = 3
sem = threading.Semaphore(num)            
url_producer = UrlProducer(sem)
url_producer.start()

got html text successgot html text successgot html text success


got html text successgot html text success

got html text success
got html text successgot html text successgot html text success


got html text successgot html text success
got html text success

got html text successgot html text successgot html text success


got html text successgot html text success

got html text success
got html text successgot html text success

